<a href="https://colab.research.google.com/github/BiggusMaximus/Tensorflow_Certification_Preparation_Dicoding/blob/main/B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===================================================================================================
# PROBLEM B4
#
# Build and train a classifier for the BBC-text dataset.
# This is a multiclass classification problem.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is originally published in: http://mlg.ucd.ie/datasets/bbc.html.
#
# Desired accuracy and validation_accuracy > 91%
# ===================================================================================================

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import pandas as pd
import csv
import numpy as np

def fit_tokenizer(train_sentences, num_words, oov_token):
    tokenizer = Tokenizer(num_words = num_words, oov_token = oov_token)
    tokenizer.fit_on_texts(train_sentences)
    return tokenizer

def seq_and_pad(sentences, tokenizer, padding, maxlen):
    sequences = tokenizer.texts_to_sequences(sentences) 
    padded_sequences = pad_sequences(sequences, padding = padding, maxlen = maxlen)
    return padded_sequences

def tokenize_labels(all_labels, split_labels):
    label_tokenizer = Tokenizer()
    label_tokenizer.fit_on_texts(all_labels)
    label_seq = np.array(label_tokenizer.texts_to_sequences(split_labels))
    label_seq_np = np.array(label_seq) - 1
    return label_seq_np

   
def solution_B4():
    bbc = pd.read_csv(
        'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/bbc-text.csv')
    bbc.to_csv('bbc-text.csv', index = False)

    sentences = []
    labels = []
    stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
    
    with open("bbc-text.csv", 'r') as csvfile:
      reader = csv.reader(csvfile, delimiter=',')
      next(reader)
      for row in reader:
        labels.append(row[0])
        sentence = row[1]
        for word in stopwords:
          token = " " + word + " "
          sentence = sentence.replace(token, " ")
        sentences.append(sentence)

    # DO NOT CHANGE THIS CODE
    # Make sure you used all of these parameters or you can not pass this test
    vocab_size = 1000
    embedding_dim = 16
    max_length = 120
    trunc_type = 'post'
    padding_type = 'post'
    oov_tok = "<OOV>"
    training_portion = .8

    # YOUR CODE HERE
    # Using "shuffle=False"
    train_sentences, test_sentences, train_labels, test_labels = train_test_split(
        sentences, 
        labels, 
        train_size=training_portion, 
        shuffle=False
    )
    # Fit your tokenizer with training data
    tokenizer =  Tokenizer(
        num_words=vocab_size, 
        oov_token=oov_tok
    )

    tokenizer.fit_on_texts(train_sentences)
    word_index = tokenizer.word_index
    train_padded_seq = seq_and_pad(train_sentences, tokenizer, padding_type, max_length)
    val_padded_seq = seq_and_pad(test_sentences, tokenizer, padding_type, max_length)

    train_label_seq = tokenize_labels(labels, train_labels)
    val_label_seq = tokenize_labels(labels, test_labels)

    

    tf.random.set_seed(123)
    model = tf.keras.Sequential([ 
        tf.keras.layers.Embedding(
            vocab_size,
            embedding_dim, 
            input_length=max_length),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) 
    model.fit(
        train_padded_seq, 
        train_label_seq, 
        epochs=30, 
        validation_data=(val_padded_seq, val_label_seq)
    )    

    # Make sure you are using "sparse_categorical_crossentropy" as a loss fuction

    return model

    # The code below is to save your model as a .h5 file.
    # It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model = solution_B4()
    model.save("model_B4.h5")
